### 손실함수(loss function)
신경망이 특정 지표를 기준으로 최적의 매개변수 값을 탐색하게 됩니다. 신경망 학습에서 사용하는 지표는 `손실함수`라고 하빈다. 이 손실 함수는 임의의 함수를 사용할 수도 있지만 일반적으로는 오차 제곱합과 교차 엔트로피 오차를 사용합니다.

### 오차 제곱합
가장 많이 쓰이는 함수는 오차제곱합입니다. 수식은 아래와 같습니다.

> E = 1/2 ∑<sub>k</sub>(y<sub>k</sub> - t<sub>k</sub>)<sup>2</sup>

y<sub>k</sub>는 신경망의 출력값, t<sub>k</sub>는 정답 레이블, k는 데이터의 차원 수를 나타냅니다. mnist로 예시를 들어보면 아래와 같습니다.
```py
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
```
t의 경우 2번째 인덱스 값이 1이므로, 2를 나타내고 있고, y의 경우 softmax function으로 각 레이블에 대한 확률을 나타내기에, 신경망이 예측한 결과는 2에 해당한다고 볼 수 있겠습니다.

In [4]:
# 오차 제곱합 구현
import numpy as np

def sum_squares_error(y, t):
    return 0.5*(np.sum((y - t)**2))

# 실제 적용(mnist 사례)
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] # 정답은 2

# 예1 : '2'일 확률이 가장 높다고 추정한 경우
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
print(sum_squares_error(np.array(y), np.array(t)))

# 예2 : '7'일 확률이 가장 높다고 추정한 경우
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print(sum_squares_error(np.array(y), np.array(t)))

# 오차 제곱합 기준으로는 첫 번재 추정 결과가(오차가 더 작기 때문에) 정답에 더 가까울 것으로 판단할 수 있다.

0.09750000000000003
0.5975


### 교차 엔트로피 오차(cross entropy)
수식은 아래와 같습니다.

> E = - ∑<sub>k</sub> t<sub>k</sub>log y<sub>k</sub>

여기에서 사용된 log의 경우 밑이 e인 자연로그(ln)을 의미합니다. 

In [5]:
# cross entropy 구현
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta)) # 아주 작은 값인 delta를 더한 이유는 np.log에 0이 들어가면 inf가 되어 연산이 불가해지기에 아주 작은 값을 더해 발산을 막는다.

# 실제 적용(mnist)
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
print(cross_entropy_error(np.array(y), np.array(t)))

y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print(cross_entropy_error(np.array(y), np.array(t)))

0.510825457099338
2.302584092994546


### Mini Batch trainning
기계학습은 훈련 데이터에 대한 손실 함수의 값을 구하고, 그 값을 최대한 줄여주는 매개변수를 찾아냅니다. 즉 훈련데이터가 100개가 있다면 그 100개의 손실 함수 값들의 합을 지표로 삼아 평가합니다. 

지금까지 데이터 하나에 대한 손실값을 생각해왔으니, 훈련 데이터 모두에 대한 손실값을 고려합니다. 이 때 교차 엔트로피의 수식은 아래와 같이 바뀝니다.

> E = - 1/N ∑<sub>n</sub> ∑<sub>k</sub> t<sub>nk</sub>log y<sub>nk</sub>

데이터가 N개라고 하면 t<sub>nk</sub>는 n번째 데이터의 k번째 값을 의미합니다. 단순히 앞선 수식에서 N개의 데이터로 확장했지만, 마지막에 N으로 나누어 정규화를 진행했습니다. N으로 나눔으로서 `평균 손실 함수`를 구하는 것입니다. 이렇게 평균을 구해 사용하면 훈련 데이터의 개수와 상관없이 항상 일관된 지표를 얻을 수 있습니다.

MNIST의 경우 데이터셋의 개수는 60000개였습니다. 모든 데이터를 대상으로 손실 함수의 값을 구하면 시간이 조금 걸리는 편입니다. 나아가 빅 데이터를 다루다 보면 수천, 수백만개의 거대한 값이 되기도 합니다. 이런 경우 데이터 일부를 추려 전체의 `근사치`로 이용할 수 있습니다. 신경망 학습에서도 훈련 데이터로부터 일부만 골라 학습을 수행합니다. 이 일부를 `Mini Batch`라고 합니다.

즉, 60000개의 데이터 중에서 일부인 100장을 무작위로 뽑아 그 100장만을 사용하여 학습하는 행위를 일컫습니다.

In [6]:
import numpy as np
from mymodules.mnist import load_mnist

# one_hot_label=True로 설정함으로, 원핫인코딩을 진행한다.(정답 위치의 원소만 1이고 나머지 원소는 0인 배열을 얻음)
(X_train, t_train), (X_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

print(X_train.shape) # 입력 데이터의 경우 784열(원래는 28x28)인 이미지 데이터
print(t_train.shape) # 정답 레이블은 10줄짜리 데이터

(60000, 784)
(60000, 10)


In [8]:
# 랜덤하게 10장만 빼오기
train_size = X_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
X_batch = X_train[batch_mask]
t_batch = t_train[batch_mask]
print(X_batch.shape) # 60000개의 데이터 중 랜덤하게 10개만 가져옴
print(t_batch.shape) # 마찬가지로 60000개의 데이터 중 랜덤하게 10개만 가져옴

(10, 784)
(10, 10)


### Mini Batch용 cross entropy error 구현하기
데이터가 하나인 경우와 데이터가 배치로 묶여 입력될 경우 모두를 처리할 수 있는 코드로 구현합니다.

In [ ]:
def cross_entropy_error(y, t):
    if y.ndim == 1: # y가 1차원이라면,
        t = t.reshape(1, t.size) # 데이터의 형상을 바꾼다.
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size # batch_size로 나눠 정규화하고, 이미지 1장당 평균의 교차 엔트로피 오차를 계산한다.